In [137]:
import os
import cohere

co = cohere.Client(os.environ["COHERE_API_KEY"])

In [138]:
my_dataset = co.datasets.create(
    name="cluster_data",
    data=open("./mahabharat.jsonl", "rb"),
    type="single-label-classification-finetune-input",
)

print(my_dataset.id)

cluster-data-xpfdd5


In [139]:
ds = co.wait(my_dataset)
print(my_dataset)
print(ds)

...
...
id='cluster-data-xpfdd5'
dataset=Dataset(id='cluster-data-xpfdd5', name='cluster_data', created_at=datetime.datetime(2024, 5, 31, 11, 2, 25, 954, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 5, 31, 11, 2, 25, 954, tzinfo=datetime.timezone.utc), dataset_type='single-label-classification-finetune-input', validation_status='validated', validation_error=None, schema_='{"name":"cohere.single_label_classification_finetune_input","type":"record","fields":[{"name":"text","type":"string"},{"name":"label","type":"string"},{"name":"is_eval","type":"boolean","default":false}]}', required_fields=None, preserve_fields=None, dataset_parts=[DatasetPart(id='59f0be89-e512-43bf-941d-a8bb02643a4b', name='mahabharat.avro', url='https://storage.googleapis.com/cohere-production-user-datasets/dataset-api-temp/30b5ef85-e023-4a52-8366-4e8ce5598652/5e56f1fd-3535-4eb7-a3cc-86143148f374/cluster-data-xpfdd5/000_mahabharat.avro?X-Goog-Algorithm=GOOG4-RSA-SHA256&X-Goog-Credential=dataset%

Starting the Classify Fine-Tuning

Create a New Fine-tuned Model

In addition to using the Web UI for fine-tuning models, customers can also kick off fine-tuning jobs programmatically using the Cohere Python SDK. This can be useful for fine-tunes that happen on a regular cadence, such as nightly jobs on newly-acquired data.

Using co.finetuning.create_finetuned_model(), you can create a fine-tuned model using either a single-label or multi-label dataset.

Examples

In [140]:
from cohere.finetuning import FinetunedModel, Settings, BaseModel

In [92]:
finetune = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="single-label-ft-2",
        settings=Settings(
            base_model=BaseModel(
                base_type="BASE_TYPE_CLASSIFICATION",
            ),
            dataset_id=my_dataset.id,
        ),
    )
)

BadRequestError: status_code: 400, body: {'message': 'finetuned_model.name already exists'}

In [ ]:
print(
    f"fine-tune ID: {finetune.finetuned_model.id}, fine-tune status: {finetune.finetuned_model.status}"
)

fine-tune ID: f3d7a982-640e-4011-93e1-7151176c0661, fine-tune status: STATUS_QUEUED


Calling a fine-tune

In [ ]:
ft = co.finetuning.get_finetuned_model(finetune.finetuned_model.id)

Classify With Fine Tune Model

In [141]:
prompt = f"""
You are tutor assistant that can give answers to the questions.

** Instructions ** 
Only provide information that lies in scope of mahabharata.

Ensure that the questions which are outside the scope of mahabharata gives message as *I am not trained to answer this question How ever you can ask me any question related to mahabharata*.
"""

In [142]:
response = co.classify(
    model="70874900-e781-4771-9bce-07850602c80f-ft", inputs=["Who is PM of India?"]
)

print("The confidence levels of the labels are: {}".format(response.classifications))

The confidence levels of the labels are: [ClassifyResponseClassificationsItem(id='6c3a862e-f103-4f77-9c40-b42a55231500', input='who is pm of india?', prediction='Mahabharata', predictions=['Mahabharata'], confidence=0.51488817, confidences=[0.51488817], labels={'Mahabharata': ClassifyResponseClassificationsItemLabelsValue(confidence=0.51488817), 'Non-Mahabharata': ClassifyResponseClassificationsItemLabelsValue(confidence=0.48511183)}, classification_type='single-label')]


Chat Fine Tuning

In [169]:
chat_dataset = co.datasets.create(
    name="chat-dataset",
    data=open("./chat_finetune_data.jsonl", "rb"),
    type="chat-finetune-input",
)
print(chat_dataset)

id='chat-dataset-kqpgwr'


In [170]:
ds = co.wait(chat_dataset)
print(ds)

...
...
dataset=Dataset(id='chat-dataset-kqpgwr', name='chat-dataset', created_at=datetime.datetime(2024, 5, 31, 12, 50, 28, 283994, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 5, 31, 12, 50, 28, 283994, tzinfo=datetime.timezone.utc), dataset_type='chat-finetune-input', validation_status='validated', validation_error=None, schema_='{"name":"cohere.chat_finetune_input","type":"record","fields":[{"name":"messages","type":{"type":"array","items":{"name":"cohere.message","type":"record","fields":[{"name":"role","type":"string"},{"name":"content","type":"string"}]}}},{"name":"is_eval","type":"boolean","default":false}]}', required_fields=None, preserve_fields=None, dataset_parts=[DatasetPart(id='a9ba24da-7142-4fbd-ba8b-350daf4005e7', name='chat_finetune_data.avro', url='https://storage.googleapis.com/cohere-production-user-datasets/dataset-api-temp/30b5ef85-e023-4a52-8366-4e8ce5598652/5e56f1fd-3535-4eb7-a3cc-86143148f374/chat-dataset-kqpgwr/000_chat_finetune_data.avro?

Fine Tune Chat Model

In [171]:
finetuned_model = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="customer-service-chat-model",
        settings=Settings(
            base_model=BaseModel(
                base_type="BASE_TYPE_CHAT",
            ),
            dataset_id=chat_dataset.id,
        ),
    ),
)

In [172]:
ft = co.finetuning.get_finetuned_model(finetuned_model.finetuned_model.id)

In [176]:
response = co.chat(
    message=prompt,
    model=ft.finetuned_model.id + "-ft",
    # optional (to specify a preamble)
    preamble="You are a chatbot trained to answer on mahabharata related questions only.",
    # optional
    return_prompt=True,
)

In [164]:
print(response.text)

Sure! I will answer your queries related to the great Indian epic, the Mahabharata. Ask away!


In [177]:
new_response = co.chat(
    message="Who is the PM of India?",
    model=ft.finetuned_model.id + "-ft",
)

In [178]:
print(new_response.text)

The current Prime Minister of India is Narendra Modi. He has held the office since 2014.
